## ERAIN Means

Monthly and Annual means of atmospheric temperatures 2 metres above sea level, averaged over the region

In [1]:
# import libraries
from datetime import datetime, timedelta
import numpy as np
import xarray
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.ticker as ticker
import pandas as pd
from pandas import Series, DataFrame
import gsw
import cmocean
import sys
import glob
import errno
import os
import os.path
import netCDF4 as nc
import geopandas as gpd
import string
import time
import csv
from matplotlib.backends.backend_pdf import PdfPages

C:\Users\maria\AppData\Local\Continuum\anaconda3\lib\site-packages\cmocean\tools.py:76: MatplotlibDeprecationWarning: The is_string_like function was deprecated in version 2.1.
  if not mpl.cbook.is_string_like(rgbin[0]):


In [62]:
datadir = '/path/ncdatafile/'
for filename in os.listdir(datadir):
    #savedir = '/path/ncdatafile/' # change this if you want to store your plots somewhere else
    #def filecount(dir_name):
    #   return len([f for f in os.listdir(dir_name) if os.path.isfile(f)])
        
    fname = 'allyearsmean.nc'
        
    ncdata = xarray.open_dataset(datadir + fname)
    juld = pd.to_datetime(ncdata.time.values) # convert time values to datetime
        
    lats = np.array(ncdata.lat.values) # latitude
    lons = np.array(ncdata.lon.values) # longitude
        
    temp = np.array(ncdata.t2m.values) # atmospheric temperature 2m above sea level in Kelvins
    names = ['x', 'y', 'z']
    index = pd.MultiIndex.from_product([range(s)for s in temp.shape], names=names)
    df = pd.DataFrame({'temp': temp.flatten()})
        
    df1 = DataFrame(juld)

    dft = pd.concat([df, df1], axis=1) # concatenate both dataframes
    dft.columns = ['temp', 'yeardate'] # Assign column headers
    df2 = dft.set_index("yeardate", drop = True)
    df3 = df2.drop_duplicates() # drop duplicate rows
        
with open("path\csvfile", "a")  as f:
    # Insert path for the file you'll save the dataframe in
    table1 = df3.to_csv(f, sep=',', header=False)

In [ ]:
ncdata.t2m

In [ ]:
df3

In [2]:
def try_parsing_date(text):
    """ This function takes in date written as a string 
    in two different formats and returns it as datetime"""
    for fmt in ('%Y-%m-%d', '%Y-%m-%d %H:%M:%S'):
        try:
            return datetime.strptime(text, fmt)
        except ValueError:
            pass
    raise ValueError('no valid date format found')

In [ ]:
# Plot monthly mean atmospheric temperatures in Kelvins
x=[]
y=[]

with open('path\csvfile','r') as csvfile:
    # Insert path of the csv file you'll plot
    plots = csv.reader(csvfile, delimiter=',')
    for column in plots:
        yeartime = column[0]
        x.append(try_parsing_date(yeartime))
        y.append(float(column[1])) # temp column

plt.scatter(x,y, label='Loaded from file!', color='k', s=25, marker="o")
plt.xlabel('Date')
plt.ylabel('Atmospheric Temperature (K)')
#plt.yscale('symlog', linthreshx=0.1)
plt.title('Monthly Mean 2m Air Temperatures')
plt.show()

In [ ]:
# Plot monthly mean atmospheric temperatures in degrees Celsius
x=[]
y=[]

with open('path\csvfile','r') as csvfile:
    # Insert path of the csv file you'll plot
    plots = csv.reader(csvfile, delimiter=',')
    for column in plots:
        yeartime = column[0]
        x.append(try_parsing_date(yeartime))
        y.append(float(column[1])-273.15) # Convert Kelvins to degrees Celsius

plt.scatter(x,y, label='Loaded from file!', color='k', s=25, marker="o")
plt.xlabel('Date')
plt.ylabel('Atmospheric Temperature (C)')
#plt.yscale('symlog', linthreshx=0.1)
plt.title('Monthly Mean 2m Air Temperatures')
plt.show()

In [ ]:
df1 = pd.read_csv('path\csvfile', header=None, error_bad_lines=False, delimiter=',')
# Insert csvfile you'll plot
df1.columns = ['Year', 'Temp2m'] # Assign column headers
df1

In [ ]:
df2 = df1.set_index('Year', drop = False)
df2

In [ ]:
#Plot Annual Mean of Monthly Means of atmospheric temperatures in Kelvins
df3 = df2['Temp2m'].groupby(df2['Year'].str[:4]).mean().rename('MEAN').reset_index()
print (df3)

fig=plt.figure(figsize=(25,10))
plt.plot(df3['Year'], df3['MEAN'])
plt.scatter(df3['Year'], df3['MEAN'])
plt.grid(alpha=0.8, linestyle=':', linewidth=.5)
plt.xlabel('Year', fontsize = 13)
plt.ylabel('Air Temperature (K)', fontsize = 13)
#plt.yscale('symlog', linthreshx=0.1)
plt.title('Annual Mean of Monthly Means of 2m Air Temperatures')
plt.show()

In [ ]:

df3 = df2['Temp2m'].groupby(df2['Year'].str[:4]).mean().rename('MEAN').reset_index()
print (df3)

fig=plt.figure(figsize=(25,10))
plt.plot(df3['Year'], df3['MEAN']-273.15) # Convert Kelvins into degrees Celsius
plt.scatter(df3['Year'], df3['MEAN']-273.15) # Convert Kelvins into degrees Celsius
plt.xlabel('Year', fontsize = 20)
plt.ylabel('Air Temperature [degree C]', fontsize = 20)
plt.grid(alpha=0.8, linestyle=':', linewidth=.5)
#plt.yscale('symlog', linthreshx=0.1)
plt.xticks(fontsize = 11)
plt.yticks(fontsize = 15)
plt.title('Annual Mean of Monthly Means of 2m Air Temperatures', fontsize = 25)
plt.show()

In [ ]:
#Plot Annual Means of atmospheric temperature with Std. Dev. in Kelvins

df5 = df2['Temp2m'].groupby(df2['Year'].str[:4]).agg(['mean', 'std', 'count']).reset_index()
print (df5)

x = df5['Year']
y = df5['mean'] # Means dataframe
e = df5['std'] # Standard deviation dataframe

plt.figure(figsize=(20,10))
plt.plot(x, y)
plt.scatter(x, y)
plt.xlabel('Year')
plt.ylabel('Air Temperature (K)')
plt.errorbar(x, y, e, linestyle='None', marker='^')
plt.title('Annual Mean Air Temperature')
plt.show()

In [ ]:
# Plot Monthly Means of atmospheric temperature in Kelvins

df4 = df2['Temp2m'].groupby(df2['Year'].str[:7]).mean().rename('MEAN').reset_index()
print (df4)

fig=plt.figure(figsize=(50,10))
plt.plot(df4['Year'], df4['MEAN'])
plt.scatter(df4['Year'], df4['MEAN'])
plt.xlabel('Year', fontsize = 13)
plt.ylabel('Air Temperatures (K)', fontsize = 13)
#plt.yscale('symlog', linthreshx=0.1)
plt.title('Monthly Means of 2m Air Temperatures')
plt.show()

In [ ]:
# Plot Monthly Means of atmospheric temperature in degrees Celsius

df4 = df2['Temp2m'].groupby(df2['Year'].str[:7]).mean().rename('MEAN').reset_index()
print (df4)

fig=plt.figure(figsize=(50,10))
plt.plot(df4['Year'], df4['MEAN']-273.15) # Convert Kelvins into degrees Celsius
plt.scatter(df4['Year'], df4['MEAN']-273.15) # Convert Kelvins into degrees Celsius
plt.xlabel('Year', fontsize = 13)
plt.ylabel('Air Temperatures (C)', fontsize = 13)
#plt.yscale('symlog', linthreshx=0.1)
plt.title('Monthly Means of 2m Air Temperatures')
plt.show()